In [ ]:
import torch
import torchvision
import numpy as np
import random
import torch.utils.data as data
import torch.nn as nn

: 

In [2]:
# Class Dictionary for CIFAR10
classDict = {'plane': 0, 'car': 1, 'bird': 2, 'cat': 3, 'deer': 4,
             'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}

binaryClasses = {0:'Machine', 1:'Animal'} # Machine , Animal

data_mean = (0.4914, 0.4822, 0.4465)
data_std = (0.2470, 0.2435, 0.2616)

In [5]:
from torch.nn.modules.transformer import TransformerDecoderLayer
# Overwrite getitem method to obtain the index of the images when iterating through the images

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader


class CIFAR10(Dataset):
    def __init__(self, train, transform):
        self.cifar10 = torchvision.datasets.CIFAR10(
                        root='./data', train=train, download=True, transform=transform)
        self.targets = self.cifar10.targets
        self.classes = self.cifar10.classes
        self.data = self.cifar10.data


    # Overloaded the getitem method to return index as well
    def __getitem__(self, index):
        data, target = self.cifar10[index]
        return data, target, index

    # Method to get all images' indices from a certain class without iterating through the loader
    def get_index(self, target_label):
      index_list = []
      for index, label in enumerate(self.targets):
        if label == target_label:
          index_list.append(index)
      return index_list

    def __len__(self):
        return len(self.cifar10)

    def remove(self, remove_list):
      mask = np.ones(len(self.cifar10), dtype=bool)
      mask[remove_list] = False
      data = self.data[mask]

# Data Prep.
inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(data_mean, data_std)],
   std= [1/s for s in data_std]
)

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(data_mean, data_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(data_mean, data_std),
])


In [6]:
trainset = CIFAR10(train=True, transform=transform_train)
testset = CIFAR10(train=False, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
class ResNet(torchvision.models.ResNet):
    """ResNet generalization for CIFAR-like thingies.

    This is a minor modification of
    https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py,
    adding additional options.
    """

    def __init__(self, block, layers, num_classes=2, zero_init_residual=False,
                 groups=1, base_width=64, replace_stride_with_dilation=[False, False, False, False],
                 norm_layer=torch.nn.BatchNorm2d, strides=[1, 2, 2, 2], initial_conv=[3, 1, 1]):
        """Initialize as usual. Layers and strides are scriptable."""
        super(torchvision.models.ResNet, self).__init__()  # torch.nn.Module
        self._norm_layer = norm_layer

        self.dilation = 1
        if len(replace_stride_with_dilation) != 4:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 4-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups

        self.inplanes = base_width
        self.base_width = 64  # Do this to circumvent BasicBlock errors. The value is not actually used.
        self.conv1 = torch.nn.Conv2d(3, self.inplanes, kernel_size=initial_conv[0],
                                     stride=initial_conv[1], padding=initial_conv[2], bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = torch.nn.ReLU(inplace=True)

        layer_list = []
        width = self.inplanes
        for idx, layer in enumerate(layers):
            layer_list.append(self._make_layer(block, width, layer, stride=strides[idx], dilate=replace_stride_with_dilation[idx]))
            width *= 2
        self.layers = torch.nn.Sequential(*layer_list)

        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.fc = torch.nn.Linear(width // 2 * block.expansion, num_classes)
        #self.predict = nn.Sigmoid()

        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (torch.nn.BatchNorm2d, torch.nn.GroupNorm)):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the arch by 0.2~0.3% according to https://arxiv.org/abs/1706.02677



    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layers(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x) # Sigmoid
        #x = self.predict(x)
        return x

initial_conv = [3, 1, 1]
NN_model = ResNet(torchvision.models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=10, base_width=64, initial_conv=initial_conv)

In [8]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
    x = torch.ones(1, device=device)
    print(x)
else:
    print("Running on a CPU...Uhh, are you sure you want to do this?")

tensor([1.], device='mps:0')


In [9]:
# Setting up training params
epochs = 21
eta = 0.01
optimizer = torch.optim.SGD(params = NN_model.parameters(), lr = eta, weight_decay = 5e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

loss_fun = nn.CrossEntropyLoss()

NN_model.to(device)
NN_model.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layers): Sequential(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3,

In [10]:
# Used for NN
DATASET = 'CIFAR10'      # Choose between 'CIFAR2', 'CIFAR10'
MODEL = 'RESNET18'       # Choose between 'RESNET18', 'VGG11'
AUGMENTS = True          # Use Data Augmentation
SAVEMODEL = False         # Save Clean Model
# LOADMODEL = False        # Load Clean Model

# Save or Load Clean Model

import os
PATH = "./crypto_hackathon/model"
os.makedirs(PATH, exist_ok = True)
PATH += "/resnet_cifar.ptr"

In [11]:
def evaluate_model(loader, model, valid_losses = [], correct = 0, total = 0):
    model.eval()

    # Evaluate Model
    for inputs, labels, index in loader:
        inputs, labels = inputs.to(device), labels.to(device)


        with torch.no_grad():
            output = model(inputs)
            if DATASET == 'CIFAR2':
                labels = labels.to(torch.float32)
                output = output.flatten()

        # negative labels: when using hinge embedding loss only
        flipped_labels = labels # * -1
        loss = loss_fun(output, flipped_labels)   # Calculate loss

        valid_loss = loss_fun(output, labels)
        valid_losses.append(valid_loss.item())

        #predictions = torch.argmax(output, dim=1)
        if DATASET == 'CIFAR2':
            predictions = torch.where(output < 0, 0, 1)
        else:
            predictions = torch.argmax(output.data, dim=1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()

    return valid_losses, correct, total


In [12]:
# if local model is not supported
NN_model.load_state_dict(torch.load("./crypto_hackathon/cuda_resnet_cifar.ptr", map_location='cpu'))
NN_model.to('mps')
#torch.save(NN_model.state_dict, PATH)

valid_losses, correct, total = evaluate_model(testloader, NN_model)
print("Valid loss: {}, Accuracy: {}".format(np.mean(valid_losses), correct / total))

Valid loss: 0.6206142711639404, Accuracy: 0.8369


In [13]:
# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import json
#import numpy as np
from sklearn.svm import SVC
import sk2torch
#import torch
import ezkl
import os

import warnings
warnings.filterwarnings('ignore')

In [14]:
async def async_function(data_path, model_path, settings_path, resource_string):
    res = await ezkl.calibrate_settings(data_path, model_path, settings_path, resource_string)
    assert res == True

In [15]:
folder = "./tmp/"

# Create the directory 'tmp' in the current working directory
try:
    os.makedirs(folder, exist_ok=True)
    print(f"Directory '{folder}' created successfully")
except OSError as error:
    print(f"Directory '{folder}' cannot be created. Error: {error}")

Directory './tmp/' created successfully


In [16]:


#pk_path = os.path.join('./tmp/test.pk')
#vk_path = os.path.join('./tmp/test.vk')
#proof_path = os.path.join('./tmp/proof.json')

def gen_witness_NN(model, img):
    model_path = os.path.join(folder, 'network.onnx')
    compiled_model_path = os.path.join(folder, 'network.compiled')
    settings_path = os.path.join(folder, 'settings.json') 
    witness_path = os.path.join(folder, 'witness.json')
    data_path = os.path.join(folder, 'input.json')
    srs_path = os.path.join(folder, 'kzg.srs')

    model = model.cpu()
    model.eval()
    x = img.cpu()

    # Export the model
    torch.onnx.export(model,                   # model being run
                    x,                         # model input (or a tuple for multiple inputs)
                    model_path,                # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=10,          # the ONNX version to export the model to
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    input_names = ['input'],   # the model's input names
                    output_names = ['output'], # the model's output names
                    dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

    data_array = ((x).detach().numpy()).reshape([-1]).tolist()

    data = dict(input_data = [data_array])

    # Serialize data into file:
    json.dump(data, open(data_path, 'w'))

    !RUST_LOG=trace
    # TODO: Dictionary outputs
    res = ezkl.gen_settings(model_path, settings_path)
    assert res == True

    res = async_function(data_path, model_path, settings_path, "resource")
    #res = await ezkl.calibrate_settings(data_path, model_path, settings_path, resource_string)
    #assert res == True

    res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
    assert res == True

    # srs path
    res = ezkl.get_srs(srs_path, settings_path)

    # now generate the witness file
    res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
    assert os.path.isfile(witness_path)

    with open(witness_path, "r") as f:
        wit = json.load(f)

    with open(settings_path, "r") as f:
        setting = json.load(f)

    prediction_array = []
    for value in wit["outputs"]:
        for field_element in value:
            prediction_array.append(ezkl.vecu64_to_float(field_element, setting['model_output_scales'][0]))
    return torch.argmax(torch.Tensor([prediction_array]), dim=1)
    #print ('Prediction:', torch.argmax(torch.Tensor([prediction_array]), dim=1) == label.cpu())

In [17]:
# classes = [2,3,5] # bird, cat, dog
from torch.utils.data import DataLoader, Subset
def gen_targetloader(dataset, target_class, num_samples = 100):
    target_index = dataset.get_index(target_class)

    if len(target_index) > num_samples:
        target_index = random.sample(target_index, num_samples)

    targetset = data.Subset(dataset, target_index)
    targetloader = torch.utils.data.DataLoader(targetset)

    return targetloader, target_index

def gen_random_subset_dataloader(dataset, num_samples = 100):
    indices = random.sample(range(len(dataset)), num_samples)
    random_subset = Subset(dataset, indices)

    return DataLoader(random_subset), indices

### Examples

In [18]:
bird_targetloader, bird_indices = gen_targetloader(testset, target_class = 2, num_samples = 100)
imgs, indices, labels = [],[],[]
for input, label, index in bird_targetloader:
    input, label = input.to(device), label.to(device)
    imgs.append(input)
    indices.append(index)
    labels.append(label)

In [17]:
NN_tmp = gen_witness_NN(NN_model, imgs[2])

In [18]:
def verify_NN_proof():
    compiled_model_path = os.path.join(folder, 'network.compiled')
    settings_path = os.path.join(folder, 'settings.json') 
    witness_path = os.path.join(folder, 'witness.json')
    proof_path = os.path.join(folder, 'proof.json')
    srs_path = os.path.join(folder, 'kzg.srs')

    pk_path = os.path.join(folder, 'test.pk')
    vk_path = os.path.join(folder, 'test.vk')

    res = ezkl.setup(
            compiled_model_path,
            vk_path,
            pk_path,
            srs_path,
        )


    assert res == True
    assert os.path.isfile(vk_path)
    assert os.path.isfile(pk_path)
    assert os.path.isfile(settings_path)

    # Generate the proof
    proof = ezkl.prove(
            witness_path,
            compiled_model_path,
            pk_path,
            proof_path,
            srs_path,
            "single",
        )
    print(proof)
    assert os.path.isfile(proof_path)

    # verify our proof
    res = ezkl.verify(
            proof_path,
            settings_path,
            vk_path,
            srs_path,
        )

    assert res == True
    print("verified")

In [19]:
verify_NN_proof()

will be using column duplication for 3778 advice columns
will be using column duplication for 3778 advice columns
will be using column duplication for 3778 advice columns
spawning module 2
will be using column duplication for 3778 advice columns
will be using column duplication for 3778 advice columns
will be using column duplication for 3778 advice columns
spawning module 2


: 